In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os
import nltk
!pip install pyspellchecker
from spellchecker import SpellChecker
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.sequence import pad_sequences


C:\Users\HP\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [3]:
file = open("nutrition.txt", "r", encoding = "utf8")

# store file in list
lines = []
for i in file:
    lines.append(i)

# Convert list to string
data = ""
for i in lines:
  data = ' '. join(lines)

#replace unnecessary stuff with space
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','').replace(',','').replace('.','.').replace(';','').replace(')','').replace('(','')  #new line, carriage return, unicode character --> replace by space

#remove unnecessary spaces
data = data.split()
data = ' '.join(data)
data[:10000]

'There are numerous things required for the existence of the individual; food and nutrition is considered as one of the imperative aspects that are essential for the survival of the human being. An individual in his life implements number of tasks and duties, all the functions are effectively carried out by obtaining energy, vigor and strength and that one obtains through acquiring food and proper nutrition. Food is not only vital to carry out ones job duties, performing well academically or putting into operation the household chores and rendering an effective contribution in other activities such as physical exercises, music, dance, arts and crafts, sports and so forth; food is considered to be an essential requirement for ones normal growth and development. The intake of certain foods called nutrients influence the functions of the body, protect the body against diseases or illnesses, reinstate health and establish people’s responses to changes that occur within an environment. Food

In [4]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

# saving the tokenizer for predict function
pickle.dump(tokenizer, open('token.pkl', 'wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:15]
len(sequence_data)
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)
sequences = []

for i in range(3, len(sequence_data)):
    words = sequence_data[i-3:i+1]
    sequences.append(words)

print("The Length of sequences are: ", len(sequences))
sequences = np.array(sequences)
sequences[:10]
X = []
y = []

for i in sequences:
    X.append(i[0:3])
    y.append(i[3])

X = np.array(X)
y = np.array(y)
print("Data: ", X[:10])
print("Response: ", y[:10])
y = to_categorical(y, num_classes=vocab_size)
y[:5]

1000
The Length of sequences are:  4094
Data:  [[ 79   9 449]
 [  9 449 450]
 [449 450  43]
 [450  43  19]
 [ 43  19   1]
 [ 19   1  87]
 [  1  87   3]
 [ 87   3   1]
 [  3   1  16]
 [  1  16   6]]
Response:  [450  43  19   1  87   3   1  16   6   2]


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [5]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=3))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 10)             10000     
                                                                 
 lstm (LSTM)                 (None, 3, 1000)           4044000   
                                                                 
 lstm_1 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense (Dense)               (None, 1000)              1001000   
                                                                 
 dense_1 (Dense)             (None, 1000)              1001000   
                                                                 
Total params: 14060000 (53.63 MB)
Trainable params: 14060000 (53.63 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("next_words.h5", monitor='loss', verbose=1, save_best_only=True)
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001))
model.fit(X, y, epochs=50, batch_size=64, callbacks=[checkpoint])

Epoch 1/50
64/64 [==============================] - ETA: 0s - loss: 6.1485
Epoch 1: loss improved from inf to 6.14847, saving model to next_words.h5


C:\Users\HP\anaconda3\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


64/64 [==============================] - 36s 436ms/step - loss: 6.1485
Epoch 2/50
64/64 [==============================] - ETA: 0s - loss: 5.7846
Epoch 2: loss improved from 6.14847 to 5.78457, saving model to next_words.h5
64/64 [==============================] - 26s 399ms/step - loss: 5.7846
Epoch 3/50
64/64 [==============================] - ETA: 0s - loss: 5.7205
Epoch 3: loss improved from 5.78457 to 5.72047, saving model to next_words.h5
64/64 [==============================] - 26s 400ms/step - loss: 5.7205
Epoch 4/50
64/64 [==============================] - ETA: 0s - loss: 5.6727
Epoch 4: loss improved from 5.72047 to 5.67272, saving model to next_words.h5
64/64 [==============================] - 26s 402ms/step - loss: 5.6727
Epoch 5/50
64/64 [==============================] - ETA: 0s - loss: 5.6430
Epoch 5: loss improved from 5.67272 to 5.64298, saving model to next_words.h5
64/64 [==============================] - 27s 422ms/step - loss: 5.6430
Epoch 6/50
64/64 [===============

In [4]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle
import speech_recognition
import pyttsx3
import pyaudio
import speech_recognition as sr
#calling spellcheck function
spell=SpellChecker()

# Load the model and tokenizer
model = load_model('next_words.h5')
tokenizer = pickle.load(open('token.pkl', 'rb'))

def Predict_Next_Words(model, tokenizer, text):

  sequence = tokenizer.texts_to_sequences([text])
  sequence = np.array(sequence)
  preds = np.argmax(model.predict(sequence))
  predicted_word = ""

  for key, value in tokenizer.word_index.items():
      if value == preds:
          predicted_word = key
          break
  print(predicted_word)
  return predicted_word

n=int(input("Enter 1 to speech and 2 to type the text:"))
if n==1:
    recognizer = sr.Recognizer()
    while True:
        try:
            with sr.Microphone() as mic:
                recognizer.adjust_for_ambient_noise(mic, duration=0.2)
                audio = recognizer.listen(mic)
                text = recognizer.recognize_google(audio)
                text = text.lower()
                print("Recognized text:", text)
                words=word_tokenize(text)
                corrected_words=[]
                for word in words:
                    corrected_word=spell.correction(word)
                    corrected_words.append(corrected_word)
                correct_words=" ".join(corrected_words)
                if correct_words=="stop":
                    print("Execution completed...")
                    break
                else:
                    try:
                        correct_words=correct_words.split(" ")
                        correct_words=correct_words[-3:]
                        print(correct_words)
                        Predict_Next_Words(model,tokenizer,correct_words)
                    except Exception as e:
                        print("Error occured: ",e)
                        continue
        except sr.UnknownValueError:
            continue
elif n==2:    
    while(True):
        text = input("Enter your line: ")
        words=word_tokenize(text)
        corrected_words=[]
        for word in words:
            corrected_word=spell.correction(word)
            corrected_words.append(corrected_word)
        correct_words=" ".join(corrected_words)
        if correct_words == "0":
            print("Execution completed.....")
            break
        else:
            try:
                correct_words = correct_words.split(" ")
                correct_words = correct_words[-3:]
                print(correct_words)
                Predict_Next_Words(model, tokenizer, correct_words)
            except Exception as e:
                print("Error occurred: ",e)
                continue
else:
    print("Enter valid choices")

Enter 1 to speech and 2 to type the text:2
Enter your line: one requires carbohydrates and
['requires', 'carbohydrates', 'and']
1/1 [==============================] - 2s 2s/step
children
Enter your line: vitamin D rich food
['i', 'rich', 'food']
1/1 [==============================] - 2s 2s/step
and
Enter your line: the human beings are
['human', 'beings', 'are']
1/1 [==============================] - 0s 49ms/step
individual
Enter your line: for survival in
['for', 'survival', 'in']
1/1 [==============================] - 0s 64ms/step
the
Enter your line: nature for what
['nature', 'for', 'what']
1/1 [==============================] - 0s 50ms/step
and
Enter your line: naure fot diet
['nature', 'fot', 'diet']
1/1 [==============================] - 0s 45ms/step
of
Enter your line: 0
Execution completed.....


In [10]:
model.save('trained_model.h5')

In [2]:
model = tf.keras.models.load_model('trained_model.h5')
model
